In [1]:
import pandas as pd
import altair as alt

In [2]:
calls = pd.read_csv("snakemake.input[0]", sep="\t")
# calls = pd.read_csv("/local/thomas/master-snakemake-workflow-sars-cov2/results/benchmarking/tables/aggregated_read_calls.tsv", sep="\t")
ordered_state = {
    "read" : "1. Read",
    "contig" : "2. Contig",
    "scaffold" : "3. Scaffold",
    "polished_scaffold" : "4. Pol. Scaffold",
    "pseudo" : "5. Pseudo"
}
calls["state"].replace(ordered_state, inplace=True)
calls

,lineage,actual_lineage,read_number,read_length,correct_call,state,sample
0,None,B.1.311,100,100,False,2. Contig,0
1,None,B.1.311,100,100,False,3. Scaffold,0
2,None,B.1.311,100,100,False,4. Pol. Scaffold,0
3,None,B.1.311,100,100,False,5. Pseudo,0
4,B.1.311,B.1.311,100,100,True,1. Read,0
...,...,...,...,...,...,...,...
145,B.1.482,B.1.482,100000,250,True,2. Contig,29
146,B.1.482,B.1.482,100000,250,True,3. Scaffold,29
147,B.1.482,B.1.482,100000,250,True,4. Pol. Scaffold,29
148,B.1.482,B.1.482,100000,250,True,5. Pseudo,29


In [3]:
sort_order = ['read', 'contig', 'scaffold', 'polished_scaffold', 'pseudo']

alt.Chart(calls).mark_bar().encode(
    y=alt.Y('read_number:O', axis=alt.Axis(format=',.0f'), title='Read Count'),
    x=alt.X('read_length:O', title='Read Length'),
    color=alt.Color('sum(correct_call):Q', title='Correctly called', legend=alt.Legend(orient='right')),
    # color=alt.Color('sum(correct_call):Q', title='Correctly called', legend=alt.Legend(orient='right', format='.0%')),
    column=alt.Column('state', title='Called on'),
    tooltip=['sum(correct_call)', 'count(correct_call)', 'fraction:Q']
).save(snakemake.output[0])

alt.Chart(...)